In [1]:
# Загрузка библиотек
import numpy as np # для работы с массивами
import pandas as pd # для работы с DataFrame 
from sklearn import datasets # для импорта данных
import seaborn as sns # для визуализации статистических данных
import matplotlib.pyplot as plt # для построения графиков


column_names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'PRICE']
boston_data = pd.read_csv('../../data/housing.csv', header=None, delimiter=r"\s+", names=column_names)
boston_data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,PRICE
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


In [4]:
boston_data[['CHAS', 'LSTAT', 'CRIM','RM']].describe()

,CHAS,LSTAT,CRIM,RM
count,506.000000,506.000000,506.000000,506.000000
mean,0.069170,12.653063,3.613524,6.284634
std,0.253994,7.141062,8.601545,0.702617
min,0.000000,1.730000,0.006320,3.561000
25%,0.000000,6.950000,0.082045,5.885500
50%,0.000000,11.360000,0.256510,6.208500
75%,0.000000,16.955000,3.677083,6.623500
max,1.000000,37.970000,88.976200,8.780000


In [5]:
# составляем матрицу А и вектор целевой переменной
i=boston_data.shape[0] #переменная для создания количества строк с единицами 
CHAS = boston_data['CHAS']
LSTAT = boston_data['LSTAT']
CRIM = boston_data['CRIM']
RM = boston_data['RM']
A = np.column_stack((np.ones(i), CHAS, LSTAT, CRIM, RM))
y = boston_data[['PRICE']]
print(A)

[[1.0000e+00 0.0000e+00 4.9800e+00 6.3200e-03 6.5750e+00]
 [1.0000e+00 0.0000e+00 9.1400e+00 2.7310e-02 6.4210e+00]
 [1.0000e+00 0.0000e+00 4.0300e+00 2.7290e-02 7.1850e+00]
 ...
 [1.0000e+00 0.0000e+00 5.6400e+00 6.0760e-02 6.9760e+00]
 [1.0000e+00 0.0000e+00 6.4800e+00 1.0959e-01 6.7940e+00]
 [1.0000e+00 0.0000e+00 7.8800e+00 4.7410e-02 6.0300e+00]]


In [6]:
# вычисляем OLS-оценку для коэффициентов без стандартизации
w_hat=np.linalg.inv(A.T@A)@A.T@y
print(w_hat.values)

[[-1.92052548]
 [ 3.9975594 ]
 [-0.58240212]
 [-0.09739445]
 [ 5.07554248]]


In [7]:
# составляем матрицу наблюдений без дополнительного столбца из единиц
A = boston_data[['CHAS', 'LSTAT', 'CRIM','RM']]
y = boston_data[['PRICE']]
# стандартизируем векторы в столбцах матрицы A
A_cent = A - A.mean(axis=0)
A_st = A_cent/np.linalg.norm(A_cent, axis=0)
A_st.describe().round(2)

,CHAS,LSTAT,CRIM,RM
count,506.00,506.00,506.00,506.00
mean,-0.00,-0.00,-0.00,-0.00
std,0.04,0.04,0.04,0.04
min,-0.01,-0.07,-0.02,-0.17
25%,-0.01,-0.04,-0.02,-0.03
50%,-0.01,-0.01,-0.02,-0.00
75%,-0.01,0.03,0.00,0.02
max,0.16,0.16,0.44,0.16


In [8]:
print(np.linalg.norm(A_st, axis=0))
## [1. 1. 1. 1.]

[1. 1. 1. 1.]


In [9]:
# стандартизируем вектор целевой переменной
y_cent = y - y.mean()
y_st = y_cent/np.linalg.norm(y_cent)

In [10]:
# вычислим OLS-оценку для стандартизированных коэффициентов
w_hat_st=np.linalg.inv(A_st.T@A_st)@A_st.T@y_st
print(w_hat_st.values)

[[ 0.11039956]
 [-0.45220423]
 [-0.09108766]
 [ 0.38774848]]


In [11]:
# матрица Грама
A_st.T @ A_st

,CHAS,LSTAT,CRIM,RM
CHAS,1.000000,-0.053929,-0.055892,0.091251
LSTAT,-0.053929,1.000000,0.455621,-0.613808
CRIM,-0.055892,0.455621,1.000000,-0.219247
RM,0.091251,-0.613808,-0.219247,1.000000


### Задание 4.3
Стандартизируйте вектор $ \vec{x}=(12, 8)^{T} $ , приведя его к единичной длине. В качестве ответа введите координаты полученного вектора. Ответ округлите до третьего знака после точки-разделителя.

$  \vec{x} = \left(\begin{array}{c} 12 \\ 8 \end{array}\right) $

$ x_{cent}=\vec{x}-x_{mean} $

$ x_{mean}=\frac{12+8}{2}=10 $

$ x_{cent}=\left(\begin{array}{c} 12 \\ 8 \end{array}\right)-10=\left(\begin{array}{c} 2 \\ -2 \end{array}\right) $

$ x_{stand}=\frac{x_{cent}}{||x_{cent}||}=\frac{1}{||x_{cent}||}*x_{cent} $

$ x_{stand}=\frac{1}{\sqrt{(2)^2+(-2)^2}}*\left(\begin{array}{c} 2 \\ -2 \end{array}\right)
 =\frac{1}{\sqrt{4+4}}*\left(\begin{array}{c} 2 \\ -2 \end{array}\right)
 =\frac{1}{\sqrt{8}}*\left(\begin{array}{c} 2 \\ -2 \end{array}\right)
 =\left(\begin{array}{c} 0.707 \\ -0.707 \end{array}\right)$

In [12]:
x=np.array([12, 8])
x_cent=x-x.mean()
x_st=x_cent/np.linalg.norm(x_cent)

x_st.round(3)

array([ 0.707, -0.707])

In [13]:
x_1 = np.array([1, 2, 6])
x_2 = np.array([3000, 1000, 2000])
np.corrcoef(x_1, x_2)

array([[ 1.        , -0.18898224],
       [-0.18898224,  1.        ]])

### Задание 4.7
Вычислите коэффициент корреляции между векторами $ \vec{v}=(5, 1, 2)^T $ и $\vec{u}=(4, 2, 8)^T $.

Ответ округлите до двух знаков после точки-разделителя.

In [15]:
v = np.array([5, 1, 2])
u = np.array([4, 2, 8])
np.corrcoef(v, u).round(2)

array([[1.  , 0.05],
       [0.05, 1.  ]])

### Задание 4.8
Составьте корреляционную матрицу для системы векторов:
$$ \vec{x}_1=(5.1, 1.8, 2.1, 10.3, 12.1, 12.6)^T $$
$$ \vec{x}_2=(10.2, 3.7, 4.1, 20.5, 24.2, 24.1)^T $$
$$ \vec{x}_3=(2.5, 0.9, 1.1, 5.1, 6.1, 6.3)^T $$
Для расчёта используйте библиотеку NumPy или Pandas.

1. Чему равен ранг полученной корреляционной матрицы?
2. Чему равен определитель полученной корреляционной матрицы? Ответ округлите до седьмого знака после точки-разделителя.

In [53]:
X_1 = np.array([5.1, 1.8, 2.1, 10.3, 12.1, 12.6])
X_2 = np.array([10.2, 3.7, 4.1, 20.5, 24.2, 24.1])
X_3 = np.array([2.5, 0.9, 1.1, 5.1, 6.1, 6.3])

In [54]:
A= np.array([X_1,X_2,X_3])
for index, i in enumerate(A):
    x_cent=i-i.mean()
    A[index]=x_cent/np.linalg.norm(x_cent)

A_cor=A @ A.T
display(A_cor)
print('1. ранг полученной корреляционной матрицы равен:',np.linalg.matrix_rank(A_cor))
print('2. определитель полученной корреляционной матрицы равен:',np.linalg.det(A_cor).round(7))

array([[1.        , 0.99925473, 0.99983661],
       [0.99925473, 1.        , 0.99906626],
       [0.99983661, 0.99906626, 1.        ]])

1. ранг полученной корреляционной матрицы равен: 3
2. определитель полученной корреляционной матрицы равен: 5e-07


In [55]:
A= np.array([X_1,X_2,X_3])

data=pd.DataFrame(A.T, columns=['X_1','X_2','X_3'],)
z=data.corr()
display(z)
print('1. ранг полученной корреляционной матрицы равен:',np.linalg.matrix_rank(z))
print('2. определитель полученной корреляционной матрицы равен:',np.linalg.det(z).round(7))

,X_1,X_2,X_3
X_1,1.000000,0.999255,0.999837
X_2,0.999255,1.000000,0.999066
X_3,0.999837,0.999066,1.000000


1. ранг полученной корреляционной матрицы равен: 3
2. определитель полученной корреляционной матрицы равен: 5e-07
